In [25]:
import pandas as pd
import sklearn as sk
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
import seaborn as sns

In [26]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV

In [27]:
df = pd.read_excel('dataset1.xlsx')

In [28]:
df.head()

,ID,gend,mar,depend,educ,self_empl,appIncome,CoappIncome,CoappIncome2,CoappIncome.1,amount_loan,amount_loan2,term_loan_amount,term_loan_amount2,cred_hist,area_prop,Banks_nearby,Salary_nearby,Target
0,1,Male,No,0,Graduate,No,5849,0.0,2924,100.0,120,120000,360,30.0,1,Urban,3,4961,Y
1,2,Male,Yes,1,Graduate,No,4583,1508.0,3045,1608.0,128,128000,360,30.0,1,Rural,0,3917,Y
2,3,Male,Yes,0,Graduate,Yes,3000,0.0,1500,100.0,66,66000,360,30.0,1,Urban,3,6284,Y
3,4,Male,Yes,0,Not Graduate,No,2583,2358.0,2470,2458.0,120,120000,360,30.0,1,Urban,3,7941,Y
4,5,Male,No,0,Graduate,No,6000,0.0,3000,100.0,141,141000,360,30.0,1,Urban,0,6114,Y


In [29]:
df.shape

(1099, 19)

### Check nans

In [30]:
df.isna().sum()

ID                    0
gend                  0
mar                   0
depend                0
educ                  0
self_empl            55
appIncome             0
CoappIncome           0
CoappIncome2          0
CoappIncome.1         0
amount_loan           0
amount_loan2          0
term_loan_amount      0
term_loan_amount2     0
cred_hist             0
area_prop             0
Banks_nearby          0
Salary_nearby         0
Target                0
dtype: int64

In [31]:
df.groupby('self_empl').count()

,ID,gend,mar,depend,educ,appIncome,CoappIncome,CoappIncome2,CoappIncome.1,amount_loan,amount_loan2,term_loan_amount,term_loan_amount2,cred_hist,area_prop,Banks_nearby,Salary_nearby,Target
self_empl,,,,,,,,,,,,,,,,,,
0,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
No,904,904,904,904,904,904,904,904,904,904,904,904,904,904,904,904,904,904
Yes,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132,132


In [32]:
df[df['self_empl'].isna()]

,ID,gend,mar,depend,educ,self_empl,appIncome,CoappIncome,CoappIncome2,CoappIncome.1,amount_loan,amount_loan2,term_loan_amount,term_loan_amount2,cred_hist,area_prop,Banks_nearby,Salary_nearby,Target
11,12,Male,Yes,2,Graduate,NaN,2500,1840.0,2170,1940.0,109,109000,360,30.0,1,Urban,5,3967,Y
19,20,Male,Yes,0,Graduate,NaN,2600,3500.0,3050,3600.0,115,115000,360,30.0,1,Urban,7,4234,Y
24,25,Male,Yes,1,Graduate,NaN,3717,2925.0,3321,3025.0,151,151000,360,30.0,1,Semiurban,0,9946,N
29,30,Female,No,2,Graduate,NaN,3750,2083.0,2916,2183.0,120,120000,360,30.0,1,Semiurban,7,4915,Y
30,31,Male,Yes,1,Graduate,NaN,4166,3369.0,3767,3469.0,201,201000,360,30.0,1,Urban,0,3681,N
95,96,Male,No,0,Graduate,NaN,6782,0.0,3391,100.0,120,120000,360,30.0,1,Urban,2,7703,Y
107,108,Male,No,0,Not Graduate,NaN,7333,0.0,3666,100.0,120,120000,360,30.0,1,Rural,6,4208,Y
111,112,Female,Yes,0,Graduate,NaN,2929,2333.0,2631,2433.0,139,139000,360,30.0,1,Semiurban,4,7681,Y
114,115,Male,No,0,Graduate,NaN,5050,0.0,2525,100.0,118,118000,360,30.0,1,Semiurban,3,6126,Y
158,159,Male,No,0,Graduate,NaN,2980,2083.0,2531,2183.0,120,120000,360,30.0,1,Rural,2,4485,Y


In [33]:
df.self_empl = df.self_empl.fillna(0)

In [34]:
df.Target.value_counts()

Y    999
N    100
Name: Target, dtype: int64

In [35]:
from sklearn.utils import resample

df_majority = df[df.Target=='Y']
df_minority = df[df.Target=='N']
df_minority_upsampled = resample(df_minority, 
                                 replace=True, 
                                 n_samples=df_majority.shape[0],
                                 random_state=123)
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
df_upsampled.Target.value_counts()

N    999
Y    999
Name: Target, dtype: int64

In [78]:
df_encoded = pd.get_dummies(df_upsampled).drop(['Target_N', 'Target_Y'], axis = 1)
df_encoded['appIncome'] = df_upsampled['appIncome']
df_encoded['CoappIncome'] = df_upsampled['CoappIncome']
df_encoded['CoappIncome2'] = df_upsampled['CoappIncome2']
df_encoded['CoappIncome.1'] = df_upsampled['CoappIncome.1']
df_encoded['amount_loan'] = df_upsampled['amount_loan']
df_encoded['amount_loan2'] = df_upsampled['amount_loan2']
df_encoded['term_loan_amount'] = df_upsampled['term_loan_amount']
df_encoded['term_loan_amount2'] = df_upsampled['term_loan_amount2']
df_encoded['Target'] = df_upsampled['Target']

## 1. Train Dataset. Baseline

In [83]:
le = LabelEncoder()
le.fit(df_encoded['Target'])

y = le.transform(df_encoded['Target'])
X = df_encoded.drop('Target', axis=1)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [85]:
%%time
clf_lr = LogisticRegression(random_state=123).fit(X_train, y_train)
clf_knn = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)
clf_dtree = DecisionTreeClassifier(random_state = 123).fit(X_train, y_train)
clf_xgb = XGBClassifier().fit(X_train, y_train)

CPU times: user 975 ms, sys: 35.1 ms, total: 1.01 s
Wall time: 171 ms


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## 2. Test Dataset

In [86]:
print("Accuracy clf_lr is", clf_lr.score(X_test,y_test))
print("Accuracy clf_dtree is", clf_dtree.score(X_test,y_test))
print("Accuracy clf_knn is", clf_knn.score(X_test,y_test))
print("Accuracy clf_xgb is", clf_xgb.score(X_test,y_test))

Accuracy clf_lr is 0.64
Accuracy clf_dtree is 0.95
Accuracy clf_knn is 0.834
Accuracy clf_xgb is 0.968


In [88]:
print(classification_report(y_test, clf_lr.predict(X_test)))
print(classification_report(y_test, clf_dtree.predict(X_test)))
print(classification_report(y_test, clf_knn.predict(X_test)))
print(classification_report(y_test, clf_xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.63      0.66      0.64       246
           1       0.65      0.62      0.64       254

    accuracy                           0.64       500
   macro avg       0.64      0.64      0.64       500
weighted avg       0.64      0.64      0.64       500

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       246
           1       1.00      0.90      0.95       254

    accuracy                           0.95       500
   macro avg       0.95      0.95      0.95       500
weighted avg       0.95      0.95      0.95       500

              precision    recall  f1-score   support

           0       0.75      0.98      0.85       246
           1       0.98      0.69      0.81       254

    accuracy                           0.83       500
   macro avg       0.87      0.84      0.83       500
weighted avg       0.87      0.83      0.83       500

              preci

### 3. Find parameters

In [89]:
clf_lr_params = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
clf_dtree_params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
clf_knn_params = {'n_neighbors':list(range(1,31)), 'weights':["uniform", "distance"]}
clf_svm_params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
clf_xgb_params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [97]:
%%time 
grid = GridSearchCV(clf_knn, clf_knn_params, cv = 3, scoring = 'roc_auc')
grid.fit(X_train, y_train)
clf_knn_best = grid.best_estimator_
print(grid.best_estimator_, grid.best_score_)
print(grid.best_params_)

grid = GridSearchCV(clf_dtree, clf_dtree_params, cv = 3, scoring = 'roc_auc')
grid.fit(X_train ,y_train)
clf_dtree_best = grid.best_estimator_
print(grid.best_estimator_, grid.best_score_)
print(grid.best_params_)

grid = GridSearchCV(clf_lr, clf_lr_params, cv = 3, scoring = 'roc_auc')
grid.fit(X_train, y_train)
clf_lr_best = grid.best_estimator_
print(grid.best_estimator_, grid.best_score_)
print(grid.best_params_)

grid = GridSearchCV(clf_xgb, clf_xgb_params, cv = 3, scoring = 'roc_auc')
grid.fit(X_train, y_train)
clf_xgb_best = grid.best_estimator_
print(grid.best_estimator_, grid.best_score_)
print(grid.best_params_)

KNeighborsClassifier(n_neighbors=24, weights='distance') 0.9995448314269803
{'n_neighbors': 24, 'weights': 'distance'}
DecisionTreeClassifier(max_leaf_nodes=82, min_samples_split=4, random_state=123) 0.9367862466444624
{'max_leaf_nodes': 82, 'min_samples_split': 4}


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTA

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTA

LogisticRegression(C=0.01, random_state=123) 0.7368463271756434
{'C': 0.01, 'penalty': 'l2'}
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None) 0.9935101112155428
{'colsample_bytree': 0.8, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6}
CPU times: user 7min 59s, sys: 5.55 s, total: 8min 5s
Wall time: 1min 13s


In [100]:
print("Accuracy clf_lr is", clf_lr_best.score(X_test,y_test))
print("Accuracy clf_dtree is", clf_dtree_best.score(X_test,y_test))
print("Accuracy clf_knn is", clf_knn_best.score(X_test,y_test))
print("Accuracy clf_xgb is", clf_xgb_best.score(X_test,y_test))

Accuracy clf_lr is 0.648
Accuracy clf_dtree is 0.956
Accuracy clf_knn is 0.776
Accuracy clf_xgb is 0.952


In [101]:
print(classification_report(y_test, clf_lr_best.predict(X_test)))
print(classification_report(y_test, clf_dtree_best.predict(X_test)))
print(classification_report(y_test, clf_knn_best.predict(X_test)))
print(classification_report(y_test, clf_xgb_best.predict(X_test)))

              precision    recall  f1-score   support

           0       0.63      0.69      0.66       246
           1       0.67      0.61      0.64       254

    accuracy                           0.65       500
   macro avg       0.65      0.65      0.65       500
weighted avg       0.65      0.65      0.65       500

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       246
           1       1.00      0.91      0.95       254

    accuracy                           0.96       500
   macro avg       0.96      0.96      0.96       500
weighted avg       0.96      0.96      0.96       500

              precision    recall  f1-score   support

           0       0.69      1.00      0.81       246
           1       1.00      0.56      0.72       254

    accuracy                           0.78       500
   macro avg       0.84      0.78      0.77       500
weighted avg       0.85      0.78      0.77       500

              preci

## Tree based models are best for my data
##  Unfortunately did not have much time for deeper grid search and scaling